In [ ]:
pip install snakeviz

In [ ]:
%load_ext snakeviz

In [1]:
%config IPCompleter.greedy=True
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
from apache_beam.runners.direct import direct_runner
from apache_beam.testing.test_stream import TestStream
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import DebugOptions
import time
import cProfile

/usr/local/google/home/srohde/Workdir/beam/sdks/python/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '


In [2]:
pipeline_options = PipelineOptions()

def printer(x):
    print(x)
    time.sleep(1)
    return x

runner = interactive_runner.InteractiveRunner(render_option='text')  # render_option could also be 'mute', 'text'
p = beam.Pipeline(runner=runner, options=pipeline_options)
pc = p | 'read' >> (TestStream().add_elements(range(100)))

squares = pc | 'Square' >> beam.Map(lambda x: x*x)
cubes = pc | 'Cube' >> beam.Map(lambda x: x**3)
result = p.run()

Running...

Using 0 cached PCollections
Executing 3 of 3 transforms.

Square produced {0, 2704, 1, 6084, 9025, ...}

read produced {19, 88, 43, 30, 44, ...}

Cube produced {85184, 300763, 110592, 216000, 681472, ...}

In [ ]:
runner = interactive_runner.InteractiveRunner(render_option='text')  # render_option could also be 'mute', 'text'
p = beam.Pipeline(runner=runner)

def string_builder(x):
    return (x % 10, x)

pcoll_init = (p | 'root' >> beam.Create(range(100000))
                | 'oom' >> beam.Map(string_builder)
                | beam.GroupByKey())

%snakeviz result=p.run()

print(result.get(pcoll_init))
#p.run()


In [ ]:
kPubsubSubscription = 'projects/google.com:clouddfe/subscriptions/srohde-taxi'
pipeline_options = PipelineOptions()
pipeline_options.view_as(StandardOptions).streaming = True
#pipeline_options.view_as(DebugOptions).add_experiment('beam_fn_api')

def printer(x):
    print(x)
    time.sleep(1)
    return x

runner = interactive_runner.InteractiveRunner(render_option='text')  # render_option could also be 'mute', 'text'
p = beam.Pipeline(runner=runner, options=pipeline_options)
pc = (p | 'read' >> beam.io.ReadFromPubSub(subscription=kPubsubSubscription)
        | 'print' >> beam.Map(printer))
pc = (p | 'read2' >> beam.io.ReadFromPubSub(subscription=kPubsubSubscription)
        | 'print2' >> beam.Map(printer))

#pcoll_init = p | beam.Create([])
#squares = pcoll_init | 'Square' >> beam.Map(lambda x: x*x)
#cubes = pcoll_init | 'Cube' >> beam.Map(lambda x: x**3)
cProfile.run('p.run()')

print(result.get(pc))